In [1]:
from collections import Counter
import os
from textblob import TextBlob as tb
import math

import nltk 
nltk.download("punkt")
nltk.download("stopwords")

from nltk.corpus import stopwords

from nltk import word_tokenize
import re

[nltk_data] Downloading package punkt to /home/paraz/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/paraz/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
def tf(word, blob):
    return blob.words.count(word) / len(blob.words)

def n_containing(word, bloblist):
    return sum(1 for blob in bloblist if word in blob.words)

def idf(word, bloblist):
    return math.log(len(bloblist) / (1 + n_containing(word, bloblist)))

def tfidf(word, blob, bloblist):
    return tf(word, blob) * idf(word, bloblist)

In [3]:
def clean_sentence(raw):
    noise_words = []
    stop_words = list(set(stopwords.words('english')))
    URL_REGEX = r"""(?i)\b((?:[a-z][\w-]+:(?:/{1,3}|[a-z0-9%])|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?<>""'']))"""
    raw = re.sub(URL_REGEX, '', raw, flags=re.MULTILINE)
    for n in noise_words:
        clean = raw.replace(n, " ")
    clean = re.sub("[^a-zA-Z]"," ", raw)
    clean = clean.lower()
    words = clean.split()
    stop_words = list(set(stopwords.words('english')))
    words = [w for w in words if w not in stop_words]
    return " ".join(words)

In [4]:
path = "_data/unlabled/"
files = os.listdir(path)

all_text = ""

for i in files:
    with open(path+i) as f:
        all_text+=clean_sentence(f.read())

In [5]:
len(all_text)

3769911

In [70]:
document = tb(all_text[:1000])
document1 = tb(clean_sentence("Fabulous rare pic of Einstein, Yukawa(1st Japanese Nobel), Wheeler (who coined black hole)&India's own Homi Bhabha on a walk in the woods."))

document1.tags

pos_dict = {}
for i in document1.tags:
    if i[1] in ['JJ', 'NN', 'JJR', 'NNS', 'NNP', 'NNPS'] and len(i[0]) > 3:
        pos_dict[i[0]] = i[1]
pos_dict            

{'bhabha': 'NN',
 'black': 'JJ',
 'einstein': 'NN',
 'fabulous': 'JJ',
 'hole': 'NN',
 'homi': 'NN',
 'india': 'NN',
 'japanese': 'JJ',
 'nobel': 'JJ',
 'rare': 'JJ',
 'wheeler': 'NN',
 'woods': 'NNS'}

In [81]:
bloblist  = [document1]
for i, blob in enumerate([document1]):
    scores = {word: tfidf(word, blob, bloblist) for word in blob.words}
    sorted_words = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    results = []
    for word, score in sorted_words:
        if word in pos_dict.keys():
            results.append((word, pos_dict[word]))
    results.sort(key=lambda x: x[1])
    for i in results:
        print(i[0])

black
nobel
fabulous
japanese
rare
homi
einstein
india
hole
wheeler
bhabha
woods
